#### This repo is refer to [here](https://colab.research.google.com/github/huggingface/blog/blob/master/notebooks/01_how_to_train.ipynb)

In [ ]:
!nvidia-smi

Wed Mar 10 14:03:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/VinBrain

/content/drive/MyDrive/VinBrain


In [3]:
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1

Uninstalling tensorflow-2.4.1:
  Successfully uninstalled tensorflow-2.4.1
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-pt6oqrw7
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-pt6oqrw7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.2MB 14.4MB/s 
     |████████████████████████████████| 890kB 66.7MB/s 
  Created wheel for transformers: filename=transformers-4.4.0.dev0-cp37-none-any.whl size=1966528 sha256=df1fe2771d49a7adc0e6fd9c204ed447bfde3ef5f49f8e635f5a776d2333cad5
  Stored in directory: /tmp/pip-ephem-wheel-cache-syrqz840/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
Successfully built transformers
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=84464b95f594744e5777d3d25dcb066edfd559ac325517d6d7c43f2f089836d7
  Stored i

#### Load pretrained model tiny bert from Huggingface

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("vblagoje/tiny_bert_7")

model = AutoModelForMaskedLM.from_pretrained("vblagoje/tiny_bert_7")

##### Architecture of Tiny Blue Bert

In [ ]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=Tr

##### Number of parameters

In [ ]:
model.num_parameters()

4416698

##### Architectuce of tiny Bert:

###### 1. Have 2 Layer Transformer Encoder

###### 2. Hidden unit: 128

#### Build Dataset

In [ ]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="./cxr.txt",
    block_size=128
)


/usr/local/lib/python3.7/dist-packages/transformers/data/datasets/language_modeling.py:128: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
dataset

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.2
)

#### Set to initialize our Trainer

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./tiny3",
    overwrite_output_dir=True,
    num_train_epochs=1500,
    per_gpu_train_batch_size=128,
    save_steps=200,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset
)

#### Tranining model

In [ ]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss
500,5.673100
1000,4.948300
1500,4.585700
2000,4.348100
2500,4.184900
3000,4.055600
3500,3.945100
4000,3.859600
4500,3.778600
5000,3.716900


CPU times: user 6h 35min 18s, sys: 3h 36min 34s, total: 10h 11min 52s
Wall time: 10h 11min 8s


TrainOutput(global_step=229500, training_loss=2.435954165922011, metrics={'train_runtime': 36668.7953, 'train_samples_per_second': 6.259, 'total_flos': 9.911494315008e+16, 'epoch': 1500.0, 'init_mem_cpu_alloc_delta': 7813315, 'init_mem_gpu_alloc_delta': 17671168, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 786047, 'train_mem_gpu_alloc_delta': 53075456, 'train_mem_cpu_peaked_delta': 17443508, 'train_mem_gpu_peaked_delta': 8433886208})

#### Save (model, tokenizer) to disk

In [ ]:
trainer.save_model("./tiny3")

In [ ]:
tokenizer.save_pretrained("./tokenizer-tiny3")

#### Check model TBB (Tiny Blue Bert) work

In [4]:
from transformers import BertForMaskedLM, PreTrainedTokenizerFast

In [10]:
model = BertForMaskedLM.from_pretrained("tiny3")

In [11]:
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=Tr

In [15]:
from transformers import AutoTokenizer

In [16]:
tokenizer = AutoTokenizer.from_pretrained("vblagoje/tiny_bert_7")

In [17]:
tokenizer

PreTrainedTokenizerFast(name_or_path='vblagoje/tiny_bert_7', vocab_size=30522, model_max_len=512, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [18]:
from transformers import pipeline

fill_mask = pipeline(
    'fill-mask',
    model = model,
    tokenizer = tokenizer
)

grountruth = "lungfield"

In [19]:
len("improving airspace consolidation in the right lower <mask> consistent with ")

75

In [20]:
fill_mask("improving airspace consolidation in the right lower [MASK] consistent with")

[{'score': 0.7667028903961182,
  'sequence': 'improving airspace consolidation in the right lower lobe consistent with',
  'token': 21833,
  'token_str': 'lobe'},
 {'score': 0.15824946761131287,
  'sequence': 'improving airspace consolidation in the right lower lung consistent with',
  'token': 11192,
  'token_str': 'lung'},
 {'score': 0.04731343686580658,
  'sequence': 'improving airspace consolidation in the right lower lobes consistent with',
  'token': 24423,
  'token_str': 'lobes'},
 {'score': 0.009581728838384151,
  'sequence': 'improving airspace consolidation in the right lower zone consistent with',
  'token': 4224,
  'token_str': 'zone'},
 {'score': 0.0036263479851186275,
  'sequence': 'improving airspace consolidation in the right lower base consistent with',
  'token': 2918,
  'token_str': 'base'}]